# TensorFlow

## グラフの作成とセッション内での実行

In [2]:
import tensorflow as tf

In [3]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

このコードは計算グラフを作るだけで、実際変数でさえ初期化されていない。このグラフを評価するには、`TensorFlow セッション`を開き、それを使用して変数を初期化し、fを評価する。

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
result

42

In [5]:
sess.close()

In [6]:
# 別のやり方1
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
result

42

In [7]:
# 別のやり方2(Jupyterはこれを推奨)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
result

42

`TensolFlow`は一般的に二つの部分に分割される。　一つ目が計算グラフを作る`構築フェーズ`、二つ目がそれを実行する`実行フェーズ`。

## グラフの管理
作成したノートは、自動的にデフォルトのグラフに追加される。

In [8]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

複数の独立したグラフを管理したい場合は、withブロックでそれを一時的にデフォルトグラフにする

In [9]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [10]:
x2.graph is graph

True

In [11]:
x2.graph is tf.get_default_graph()

False

## ノード値のライフサイクル

In [12]:
w = tf.constant(3)

x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


## TensorFlowによる線形回帰

In [13]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [14]:
housing = fetch_california_housing()
m, n = housing.data.shape

In [15]:
m, n

(20640, 8)

In [16]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data] # バイアス項1を付与
housing_data_plus_bias[:2]

array([[ 1.00000000e+00,  8.32520000e+00,  4.10000000e+01,
         6.98412698e+00,  1.02380952e+00,  3.22000000e+02,
         2.55555556e+00,  3.78800000e+01, -1.22230000e+02],
       [ 1.00000000e+00,  8.30140000e+00,  2.10000000e+01,
         6.23813708e+00,  9.71880492e-01,  2.40100000e+03,
         2.10984183e+00,  3.78600000e+01, -1.22220000e+02]])

In [17]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

In [18]:
XT = tf.transpose(X)

In [19]:
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [20]:
with tf.Session() as sess:
    theta_value = theta.eval()
theta_value

array([[-3.6894890e+01],
       [ 4.3661433e-01],
       [ 9.4453208e-03],
       [-1.0704148e-01],
       [ 6.4345831e-01],
       [-3.9632569e-06],
       [-3.7880042e-03],
       [-4.2093179e-01],
       [-4.3400639e-01]], dtype=float32)

NumPyで正規方程式を計算するのに比べTensorFlowはGPUが使える場合はGPUでこれを実行してくれる

## 勾配下降法の実装
正規方程式ではなく、バッチ勾配下降法を使用してみる。

### マニュアルの勾配計算
まず、手動で勾配を計算する

In [21]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

勾配下降法を使用するときには、まず入力特徴量行列を正規化することが大切だ。正規化を行わないと訓練が大幅に遅くなる。  
scikit-learnのStandardScalerを使用して正規化を行なった

In [22]:
n_epochs = 1000
learning_rate = 0.01

In [23]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')

In [24]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta') # thetaをランダムな値に設定

In [27]:
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')# 二乗誤差を計算

tf.assign()は変数に新しい値を代入するノードを作成する。  
この場合は、バッチ勾配下降法のステップ$\theta^{(next step)} = \theta - \eta\nabla_\theta MSE(\theta)$を実装している

In [28]:
gradients = 2/m * tf.matmul(tf.transpose(X), error) # 勾配を計算
training_op = tf.assign(theta, theta - learning_rate * gradients) 

In [29]:
init = tf.global_variables_initializer()

In [31]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE =', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE = 9.009025
Epoch 100 MSE = 0.7566747
Epoch 200 MSE = 0.6031534
Epoch 300 MSE = 0.5814728
Epoch 400 MSE = 0.5672543
Epoch 500 MSE = 0.5567557
Epoch 600 MSE = 0.54894036
Epoch 700 MSE = 0.54309905
Epoch 800 MSE = 0.5387155
Epoch 900 MSE = 0.535412


In [32]:
best_theta

array([[ 2.0685523e+00],
       [ 7.1603501e-01],
       [ 1.2321315e-01],
       [-2.7499818e-03],
       [ 6.7215137e-02],
       [-1.4059452e-03],
       [-3.7281755e-02],
       [-9.7401929e-01],
       [-9.2912388e-01]], dtype=float32)